In [2]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import time
import shutil
import pathlib
import itertools
from PIL import Image

import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization ,GlobalAveragePooling2D
from tensorflow.keras import regularizers

import warnings
warnings.filterwarnings("ignore")

print ('modules loaded')

modules loaded


In [7]:
import splitfolders
splitfolders.ratio(
    "/kaggle/input/ct-kidney-dataset-normal-cyst-tumor-and-stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
   output="./dataset",
   seed=7,
   ratio=(0.7,0.15, 0.15)
)

Copying files: 12446 files [02:31, 82.09 files/s] 


In [8]:
train_dir = './dataset/train'
test_dir = './dataset/test'
val_dir = './dataset/val'

train_classes = os.listdir(train_dir)
test_classes = os.listdir(test_dir)
val_classes = os.listdir(val_dir)

print(f"Classes found in training directory: {train_classes}")
print(f"Classes found in training directory: {test_classes}")
print(f"Classes found in validation directory: {val_classes}")


Classes found in training directory: ['Tumor', 'Cyst', 'Stone', 'Normal']
Classes found in training directory: ['Tumor', 'Cyst', 'Stone', 'Normal']
Classes found in validation directory: ['Tumor', 'Cyst', 'Stone', 'Normal']


In [9]:
train_datagen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    vertical_flip=True,
    horizontal_flip=True,
    fill_mode="reflect"
)

train_generator = train_datagen.flow_from_directory(
    './dataset/train',
    target_size=(224, 224),
    class_mode='categorical', 
    batch_size=64,
)

test_datagen = ImageDataGenerator(rescale = 1/255.0)

test_generator = test_datagen.flow_from_directory(
     './dataset/test',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size = 64
)

val_datagen = ImageDataGenerator(rescale=1/255.0)

val_generator = val_datagen.flow_from_directory(
    './dataset/val',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=64,
)

Found 8710 images belonging to 4 classes.
Found 1871 images belonging to 4 classes.
Found 1865 images belonging to 4 classes.


In [10]:
# Define a function to compile and train each model with uniform settings
def compile_and_train(model, model_name, epochs=20):
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    print(f"Training {model_name} Model")
    
    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=val_generator
    )
    return history

In [11]:
# Model Architectures
from tensorflow.keras.applications import ResNet50, InceptionV3, VGG16, MobileNetV2
import keras_tuner as kt
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [19]:
# 1) VGG16

In [24]:
class VGG16HyperModel(kt.HyperModel):
    def build(self, hp):
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        base_model.trainable = False  # Freezing the base model layers

        model = Sequential([base_model, GlobalAveragePooling2D()])

        # Adding Dense Layers with Hyperparameter Tuning
        model.add(Dense(units=hp.Int('units_1', min_value=256, max_value=1024, step=128),
                        activation='relu', kernel_regularizer=l2(hp.Choice('l2_strength', [1e-4, 1e-3, 1e-2]))))
        model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)))

        model.add(Dense(units=hp.Int('units_2', min_value=128, max_value=512, step=64),
                        activation='relu', kernel_regularizer=l2(hp.Choice('l2_strength_2', [1e-4, 1e-3, 1e-2]))))
        model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)))

        model.add(Dense(4, activation='softmax'))

        # Compile the model
        learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        
        return model


In [25]:
# Hyperparameter Tuning with Keras Tuner
tuner = kt.RandomSearch(
    VGG16HyperModel(),
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='vgg16_tuning',
    project_name='VGG16_hyperparameter_tuning'
)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Search for the best hyperparameters
tuner.search(train_generator,
             epochs=20,
             validation_data=val_generator,
             callbacks=[early_stopping])


Trial 4 Complete [01h 21m 54s]
val_accuracy: 0.8343163430690765

Best val_accuracy So Far: 0.8678284287452698
Total elapsed time: 04h 41m 05s


In [27]:
# Retrieve the best model
best_vgg16_model = tuner.get_best_models(num_models=1)[0]
best_vgg16hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# Summary of the best model
best_vgg16_model.summary()
print("Best hyperparameters:", best_vgg16hyperparameters.values)

print("Best VGG16 Model Performance:")

train_loss, train_accuracy = best_vgg16_model.evaluate(train_generator)
print(f"Train Accuracy: {train_accuracy:.4f}, Train Loss: {train_loss:.4f}")

val_loss, val_accuracy = best_vgg16_model.evaluate(val_generator)
print(f"Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {val_loss:.4f}")

test_loss, test_accuracy = best_vgg16_model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 384)            │       393,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │         1,540 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,635,140 (59.64 MB)

 Trainable params: 920,452 (3.51 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

Best hyperparameters: {'units_1': 1024, 'l2_strength': 0.001, 'dropout_1': 0.4, 'units_2': 384, 'l2_strength_2': 0.001, 'dropout_2': 0.2, 'learning_rate': 0.001}
Best VGG16 Model Performance:
137/137 ━━━━━━━━━━━━━━━━━━━━ 114s 824ms/step - accuracy: 0.8409 - loss: 0.5979
Train Accuracy: 0.8420, Train Loss: 0.5903
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 258ms/step - accuracy: 0.8841 - loss: 0.5218
Validation Accuracy: 0.8740, Validation Loss: 0.5441
30/30 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.8314 - loss: 0.6235 
Test Accuracy: 0.8381, Test Loss: 0.6006


In [11]:
# 2) Resnet50

In [13]:
class ResNet50HyperModel(kt.HyperModel):
    def build(self, hp):
        # Load the base ResNet50 model with pretrained weights
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        base_model.trainable = False  # Freeze the base model layers

        model = Sequential([
            base_model,
            GlobalAveragePooling2D()  # Add pooling layer
        ])

        # Adding Dense Layers with Hyperparameter Tuning
        model.add(Dense(units=hp.Int('units_1', min_value=256, max_value=1024, step=128),
                        activation='relu', kernel_regularizer=l2(hp.Choice('l2_strength', [1e-4, 1e-3, 1e-2]))))
        model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)))

        model.add(Dense(units=hp.Int('units_2', min_value=128, max_value=512, step=64),
                        activation='relu', kernel_regularizer=l2(hp.Choice('l2_strength_2', [1e-4, 1e-3, 1e-2]))))
        model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)))

        model.add(Dense(4, activation='softmax'))  # Output layer for 4 classes

        # Compile the model with tunable learning rate
        learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        
        return model

In [14]:
# Hyperparameter Tuning with Keras Tuner
tuner = kt.RandomSearch(
    ResNet50HyperModel(),
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=2,
    directory='resnet50_tuning',
    project_name='ResNet50_hyperparameter_tuning'
)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Assuming train_generator and val_generator are already defined
tuner.search(train_generator,
             epochs=20,
             validation_data=val_generator,
             callbacks=[early_stopping])

Trial 4 Complete [01h 23m 32s]
val_accuracy: 0.6278820335865021

Best val_accuracy So Far: 0.6278820335865021
Total elapsed time: 04h 58m 52s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
896               |640               |units_1
0.0001            |0.0001            |l2_strength
0.2               |0.4               |dropout_1
128               |320               |units_2
0.001             |0.01              |l2_strength_2
0.2               |0.3               |dropout_2
0.01              |0.001             |learning_rate

Epoch 1/20
 86/137 ━━━━━━━━━━━━━━━━━━━━ 43s 859ms/step - accuracy: 0.3536 - loss: 3.9108

KeyboardInterrupt: 

In [15]:
# Retrieve the best model
best_resnet50_model = tuner.get_best_models(num_models=1)[0]
best_resnethyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# Summary of the best model
best_resnet50_model.summary()

print("Best hyperparameters:", best_resnethyperparameters.values)

print("Best Resresnet50 Model Performance:")

train_loss, train_accuracy = best_resnet50_model.evaluate(train_generator)
print(f"Train Accuracy: {train_accuracy:.4f}, Train Loss: {train_loss:.4f}")

val_loss, val_accuracy = best_resnet50_model.evaluate(val_generator)
print(f"Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {val_loss:.4f}")

test_loss, test_accuracy = best_resnet50_model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 640)            │     1,311,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 320)            │       205,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 320)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │         1,284 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,105,476 (95.77 MB)

 Trainable params: 1,517,764 (5.79 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

Best hyperparameters: {'units_1': 640, 'l2_strength': 0.0001, 'dropout_1': 0.4, 'units_2': 320, 'l2_strength_2': 0.01, 'dropout_2': 0.30000000000000004, 'learning_rate': 0.001}
Best Resresnet50 Model Performance:
137/137 ━━━━━━━━━━━━━━━━━━━━ 114s 800ms/step - accuracy: 0.6021 - loss: 1.0161
Train Accuracy: 0.6052, Train Loss: 1.0126
30/30 ━━━━━━━━━━━━━━━━━━━━ 9s 281ms/step - accuracy: 0.6267 - loss: 0.9820
Validation Accuracy: 0.6306, Validation Loss: 0.9770
30/30 ━━━━━━━━━━━━━━━━━━━━ 28s 914ms/step - accuracy: 0.6031 - loss: 1.0253
Test Accuracy: 0.6050, Test Loss: 1.0217


In [ ]:
# 3) InceptionV3

In [ ]:
class InceptionV3HyperModel(HyperModel):
    def build(self, hp):
        # Load the base InceptionV3 model with pretrained weights
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        base_model.trainable = False  # Freeze the base model layers

        model = Sequential([
            base_model,
            GlobalAveragePooling2D()  # Add pooling layer
        ])

        # Adding Dense Layers with Hyperparameter Tuning
        model.add(Dense(units=hp.Int('units_1', min_value=256, max_value=1024, step=128),
                        activation='relu', kernel_regularizer=l2(hp.Choice('l2_strength', [1e-4, 1e-3, 1e-2]))))
        model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)))

        model.add(Dense(units=hp.Int('units_2', min_value=128, max_value=512, step=64),
                        activation='relu', kernel_regularizer=l2(hp.Choice('l2_strength_2', [1e-4, 1e-3, 1e-2]))))
        model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)))

        model.add(Dense(4, activation='softmax'))  # Output layer for 4 classes

        # Compile the model with tunable learning rate
        learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        
        return model


In [ ]:
# Hyperparameter Tuning with Keras Tuner
tuner = RandomSearch(
    InceptionV3HyperModel(),
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='inceptionv3_tuning',
    project_name='InceptionV3_hyperparameter_tuning'
)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Assuming train_generator and val_generator are already defined
tuner.search(train_generator,
             epochs=20,
             validation_data=val_generator,
             callbacks=[early_stopping])


In [ ]:
# Retrieve the best model
best_inceptionv3_model = tuner.get_best_models(num_models=1)[0]
best_incepthyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# Summary of the best model
best_inceptionv3_model.summary()

print("Best hyperparameters:", best_incepthyperparameters.values)

print("Best InceptionV3 Model Performance:")

train_loss, train_accuracy = best_inceptionv3_model.evaluate(train_generator)
print(f"Train Accuracy: {train_accuracy:.4f}, Train Loss: {train_loss:.4f}")

val_loss, val_accuracy = best_inceptionv3_model.evaluate(val_generator)
print(f"Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {val_loss:.4f}")

test_loss, test_accuracy = best_inceptionv3_model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")

In [ ]:
# 4) MobileNetV2

In [ ]:
class MobileNetV2HyperModel(kt.HyperModel):
    def build(self, hp):
        # Load the base MobileNetV2 model with pretrained weights
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        base_model.trainable = False  # Freeze the base model layers

        model = Sequential([
            base_model,
            GlobalAveragePooling2D()  # Add pooling layer
        ])

        # Adding Dense Layers with Hyperparameter Tuning
        model.add(Dense(units=hp.Int('units_1', min_value=256, max_value=1024, step=128),
                        activation='relu', kernel_regularizer=l2(hp.Choice('l2_strength', [1e-4, 1e-3, 1e-2]))))
        model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)))

        model.add(Dense(units=hp.Int('units_2', min_value=128, max_value=512, step=64),
                        activation='relu', kernel_regularizer=l2(hp.Choice('l2_strength_2', [1e-4, 1e-3, 1e-2]))))
        model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)))

        model.add(Dense(4, activation='softmax'))  # Output layer for 4 classes

        # Compile the model with tunable learning rate
        learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        
        return model


In [ ]:
# Hyperparameter Tuning with Keras Tuner
tuner = kt.RandomSearch(
    MobileNetV2HyperModel(),
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=2,
    directory='mobilenetv2_tuning',
    project_name='MobileNetV2_hyperparameter_tuning'
)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Assuming train_generator and val_generator are already defined
tuner.search(train_generator,
             epochs=20,
             validation_data=val_generator,
             callbacks=[early_stopping])


In [ ]:
# Retrieve the best model and its parameters
best_mobilenetv2_model = tuner.get_best_models(num_models=1)[0]
best_mobilehyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# Summarize the best model
best_mobilenetv2_model.summary()
print("Best hyperparameters:", best_mobilehyperparameters.values)

print("Best MobileNetV2 Model Performance:")

train_loss, train_accuracy = best_mobilenetv2_model.evaluate(train_generator)
print(f"Train Accuracy: {train_accuracy:.4f}, Train Loss: {train_loss:.4f}")

val_loss, val_accuracy = best_mobilenetv2_model.evaluate(val_generator)
print(f"Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {val_loss:.4f}")

test_loss, test_accuracy = best_mobilenetv2_model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")


In [12]:
# 5) LeNet

In [13]:
class LeNetHyperModel(kt.HyperModel):
    def build(self, hp):
        model = Sequential()

        # Convolutional and Pooling Layers
        model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(224, 224, 3)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, kernel_size=(5, 5), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten())

        # Tunable Dense Layers
        model.add(Dense(units=hp.Int('units_1', min_value=120, max_value=512, step=64),
                        activation='relu',
                        kernel_regularizer=l2(hp.Choice('l2_strength', [1e-4, 1e-3, 1e-2]))))
        model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)))

        model.add(Dense(units=hp.Int('units_2', min_value=84, max_value=256, step=32),
                        activation='relu',
                        kernel_regularizer=l2(hp.Choice('l2_strength_2', [1e-4, 1e-3, 1e-2]))))
        model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)))

        # Output Layer for 4 Classes
        model.add(Dense(4, activation='softmax'))

        # Compile the model with a tunable learning rate
        learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        return model

In [14]:
# Hyperparameter Tuning with Keras Tuner
tuner = kt.RandomSearch(
    LeNetHyperModel(),
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=2,
    directory='lenet_tuning',
    project_name='LeNet_hyperparameter_tuning'
)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Assuming train_generator and val_generator are already defined
tuner.search(train_generator,
             epochs=20,
             validation_data=val_generator,
             callbacks=[early_stopping])


Trial 4 Complete [01h 04m 27s]
val_accuracy: 0.7576407492160797

Best val_accuracy So Far: 0.7831099331378937
Total elapsed time: 05h 13m 26s


In [15]:
# Retrieve the best model
best_lenet_model = tuner.get_best_models(num_models=1)[0]
best_lenethyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# Summary of the best model
best_lenet_model.summary()
print("Best hyperparameters:", best_lenethyperparameters.values)

print("Best LeNet Model Performance:")

train_loss, train_accuracy = best_lenet_model.evaluate(train_generator)
print(f"Train Accuracy: {train_accuracy:.4f}, Train Loss: {train_loss:.4f}")

val_loss, val_accuracy = best_lenet_model.evaluate(val_generator)
print(f"Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {val_loss:.4f}")

test_loss, test_accuracy = best_lenet_model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 220, 220, 32)   │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 110, 110, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 106, 106, 64)   │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 53, 53, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 179776)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 248)            │    44,584,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 248)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 116)            │        28,884 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 116)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           468 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,667,744 (170.39 MB)

 Trainable params: 44,667,744 (170.39 MB)

 Non-trainable params: 0 (0.00 B)

Best hyperparameters: {'units_1': 248, 'l2_strength': 0.001, 'dropout_1': 0.2, 'units_2': 116, 'l2_strength_2': 0.0001, 'dropout_2': 0.2, 'learning_rate': 0.0001}
Best LeNet Model Performance:
137/137 ━━━━━━━━━━━━━━━━━━━━ 116s 840ms/step - accuracy: 0.7921 - loss: 0.6318
Train Accuracy: 0.7914, Train Loss: 0.6398
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 251ms/step - accuracy: 0.8210 - loss: 0.5712
Validation Accuracy: 0.8118, Validation Loss: 0.5866
30/30 ━━━━━━━━━━━━━━━━━━━━ 26s 864ms/step - accuracy: 0.7695 - loss: 0.6703
Test Accuracy: 0.7835, Test Loss: 0.6414
